In [1]:
!pip install SQLAlchemy pandas keras pymysql cryptography scikit-learn numpy matplotlib tensorflow

In [2]:
!pip install --upgrade tensorflow keras

  Using cached keras-3.0.2-py3-none-any.whl.metadata (4.8 kB)


In [3]:
# importing libs for ml model
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler

from sqlalchemy import create_engine
from pyspark.sql import SparkSession

2024-01-17 23:12:03.459955: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-17 23:12:03.500947: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-17 23:12:03.700347: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-17 23:12:03.700392: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-17 23:12:03.737260: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [4]:
# connecting with mysql
MYSQL_USER = os.environ.get('MYSQL_USER', 'root')
MYSQL_PASSWORD = os.environ.get('MYSQL_PASSWORD', 'secret')
MYSQL_DATABASE = os.environ.get('MYSQL_DATABASE', 'mydatabase')
MYSQL_DC_HOSTNAME = os.environ.get('MYSQL_DC_HOSTNAME', 'bigdata-mysql')

URL_CONNECT = f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_DC_HOSTNAME}:3306/{MYSQL_DATABASE}'

MYSQL_CONNECTION = create_engine(URL_CONNECT)

In [5]:
# --------------------------- Test

In [6]:
# query = "SELECT * FROM stock_data limit 1439833;"
# df = pd.read_sql_query(query, con=MYSQL_CONNECTION)
# df

In [7]:
# -------------------------- End Test

In [8]:
#
# Configuring Spark to execute queries
#

mysql_connector_path = "./mysql-connector-j-8.1.0.jar"
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--jars {mysql_connector_path} --master spark://bigdata-spark:7077 pyspark-shell'

mysql_url = "jdbc:mysql://bigdata-mysql:3306/mydatabase"
mysql_properties = {
    "driver": "com.mysql.cj.jdbc.Driver",
    "user": "user",
    "password": "secret"
}

In [9]:
#
# Spark - Creating session
#
spark = SparkSession.builder.appName("MySQLDataRead").getOrCreate()

24/01/17 23:12:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [10]:
# Reading data by MySQL and converting to a spark dataframe
base = spark.read \
    .format("jdbc") \
    .option("url", mysql_url) \
    .option("dbtable", "stock_data") \
    .option("user", mysql_properties["user"]) \
    .option("password", mysql_properties["password"]) \
    .option("driver", mysql_properties["driver"]) \
    .load()

# Show the spark dataframe
base.show()

# Finish this spark session
# spark.stop()

+----------+------------------+-------+------+------------------+------------------+------------------+-------+---------+------------+
|      Date|              Open|Country|Ticker|              High|               Low|             Close| Volume|Dividends|Stock Splits|
+----------+------------------+-------+------+------------------+------------------+------------------+-------+---------+------------+
|2007-09-26| 4.605397613267905| brazil| ABCB4| 4.684622764587402| 4.515838979347431| 4.684622764587402|  72665|      0.0|         0.0|
|2007-09-27| 4.605397613267905| brazil| ABCB4| 4.684622764587402| 4.515838979347431| 4.684622764587402|  72665|      0.0|         0.0|
|2007-09-28| 4.753513813018799| brazil| ABCB4|   4.9601887922053| 4.753513813018799| 4.753513813018799| 442102|      0.0|         0.0|
|2007-10-01| 4.684622519848403| brazil| ABCB4| 4.942965817868213| 4.684622519848403| 4.805182933807373| 318368|      0.0|         0.0|
|2007-10-02| 4.801737099270801| brazil| ABCB4| 4.818960

In [11]:
# -------------------------- Model

In [12]:
# Assuming that 'base' is already loaded using Spark DataFrame
# You can replace the code below with your Spark DataFrame operations

# Assuming 'Open' column contains the stock prices
base_treinamento = base.select('Open').toPandas().dropna()

In [13]:
# Feature scaling
normalizador = MinMaxScaler(feature_range=(0, 1))
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)

In [14]:
# Creating input sequences and labels
previsores = []
preco_real = []

for i in range(90, len(base_treinamento_normalizada)):
    previsores.append(base_treinamento_normalizada[i-90:i, 0])
    preco_real.append(base_treinamento_normalizada[i, 0])

previsores, preco_real = np.array(previsores), np.array(preco_real)
previsores = np.reshape(previsores, (previsores.shape[0], previsores.shape[1], 1))

In [15]:
# Building the LSTM model
regressor = Sequential()
regressor.add(LSTM(units=100, return_sequences=True, input_shape=(previsores.shape[1], 1)))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50))
regressor.add(Dropout(0.3))

regressor.add(Dense(units=1, activation='linear'))

In [16]:
regressor.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mean_absolute_error'])

In [17]:
# regressor.fit(previsores, preco_real, epochs=100, batch_size=32)

In [18]:
# Specify batch size
batch_size = 32

# Calculate the number of batches
num_batches = len(preco_real) // batch_size
num_batches = 2

# Training the model in batches
epochs = 100  # You can adjust the number of epochs
counter = 0
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size

    # Extract batch data
    batch_previsores = previsores[start_idx:end_idx]
    batch_preco_real = preco_real[start_idx:end_idx]

    # Train the model on the current batch
    regressor.fit(batch_previsores, batch_preco_real, epochs=epochs, batch_size=batch_size)

    counter+=1

    print(f'\n\n\nCounter = {counter}')
    print(f'\n\n\nNum_batches = {num_batches}')

Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 0.0034 - mean_absolute_error: 0.0583
Epoch 2/100
1/1 [==============================] - 0s 76ms/step - loss: 1.3872e-04 - mean_absolute_error: 0.0099
Epoch 3/100
1/1 [==============================] - 0s 74ms/step - loss: 1.0201e-04 - mean_absolute_error: 0.0087
Epoch 4/100
1/1 [==============================] - 0s 70ms/step - loss: 8.4791e-05 - mean_absolute_error: 0.0067
Epoch 5/100
1/1 [==============================] - 0s 73ms/step - loss: 7.1947e-05 - mean_absolute_error: 0.0065
Epoch 6/100
1/1 [==============================] - 0s 74ms/step - loss: 6.9289e-05 - mean_absolute_error: 0.0066
Epoch 7/100
1/1 [==============================] - 0s 80ms/step - loss: 5.2880e-05 - mean_absolute_error: 0.0055
Epoch 8/100
1/1 [==============================] - 0s 69ms/step - loss: 7.3410e-05 - mean_absolute_error: 0.0072
Epoch 9/100
1/1 [==============================] - 0s 91ms/step - loss: 5.6710e-05 - mean_absolute_err

In [19]:
# Preparing test data
base_teste = spark.read \
    .format("jdbc") \
    .option("url", mysql_url) \
    .option("dbtable", "stock_data") \
    .option("user", mysql_properties["user"]) \
    .option("password", mysql_properties["password"]) \
    .option("driver", mysql_properties["driver"]) \
    .load()

preco_real_teste = base_teste.select('Open').toPandas().iloc[:, 0].values

In [20]:
base_completa = pd.concat((base_treinamento['Open'], preco_real_teste), axis=0)
entradas = base_completa[len(base_completa) - len(preco_real_teste) - 90:].values
entradas = entradas.reshape(-1, 1)
entradas = normalizador.transform(entradas)

X_teste = []

for i in range(90, len(entradas)):
    X_teste.append(entradas[i-90:i, 0])

X_teste = np.array(X_teste)
X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))

TypeError: cannot concatenate object of type '<class 'numpy.ndarray'>'; only Series and DataFrame objs are valid

In [ ]:
# Making predictions
previsoes = regressor.predict(X_teste)
previsoes = normalizador.inverse_transform(previsoes)

# Plotting the results
plt.plot(preco_real_teste, color='red', label='Preço real')
plt.plot(previsoes, color='blue', label='Previsões')
plt.title('Previsão preço das ações')
plt.xlabel('Tempo')
plt.ylabel('Valor Yahoo')
plt.legend()
plt.show()